In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [2]:
import pandas as pd
from datetime import date

import os

In [3]:
import authorization as auth

In [3]:
consumer_key = os.getenv("ETRADE_SANDBOX_KEY")
consumer_secret = os.getenv("ETRADE_SANDBOX_SECRET")

In [8]:
sb_oauth = auth.ETradeOAuth(consumer_key, consumer_secret)

In [10]:
from rauth import OAuth1Service
import webbrowser

In [5]:
service = OAuth1Service(
              name = 'etrade',
              consumer_key = consumer_key,
              consumer_secret = consumer_secret,
              request_token_url = 'https://apisb.etrade.com/oauth/request_token',
              access_token_url = 'https://apisb.etrade.com/oauth/access_token',
              authorize_url = 'https://us.etrade.com/e/t/etws/authorize?key={}&token={}',
              base_url = 'https://apisb.etrade.com')


In [86]:
oauth_token, oauth_token_secret = service.get_request_token(params = 
                                  {'oauth_callback': 'oob', 
                                   'format': 'json'})

In [87]:
auth_url = service.authorize_url.format(consumer_key, oauth_token)

In [88]:
# Get verifier (direct input in console, still working on callback)
webbrowser.open(auth_url)
verifier = input('Please input the verifier: ')

Please input the verifier:  OPZHC


In [89]:
session = service.get_auth_session(oauth_token, oauth_token_secret, params = {'oauth_verifier': verifier})

In [91]:
list_accounts_url = 'https://apisb.etrade.com/v1/accounts/list.json'

In [92]:
resp = session.get(list_accounts_url, params = {'format': 'json'})

In [94]:
# resp.text
resp.json()

{'AccountListResponse': {'Accounts': {'Account': [{'accountId': '823145980',
     'accountIdKey': 'dBZOKt9xDrtRSAOl4MSiiA',
     'accountMode': 'IRA',
     'accountDesc': 'Brokerage',
     'accountName': 'NickName-1',
     'accountType': 'MARGIN',
     'institutionType': 'BROKERAGE',
     'accountStatus': 'ACTIVE',
     'closedDate': 0,
     'shareWorksAccount': False,
     'fcManagedMssbClosedAccount': False},
    {'accountId': '583156360',
     'accountIdKey': 'vQMsebA1H5WltUfDkJP48g',
     'accountMode': 'CASH',
     'accountDesc': 'Complete Savings',
     'accountName': 'NickName-2',
     'accountType': 'INDIVIDUAL',
     'institutionType': 'BROKERAGE',
     'accountStatus': 'ACTIVE',
     'closedDate': 0,
     'shareWorksAccount': False,
     'fcManagedMssbClosedAccount': False},
    {'accountId': '707004180',
     'accountIdKey': '6_Dpy0rmuQ9cu9IbTfvF2A',
     'accountMode': 'CASH',
     'accountDesc': 'INDIVIDUAL',
     'accountName': 'NickName-3',
     'accountType': 'INDIVIDUA

In [95]:
accountIdKey = "xj1Dc18FTqWPqkEEVUr5rw"
instType = "BROKERAGE"

In [96]:
account_balances_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/balance.json"

In [75]:
account_balances_url

'https://apisb.etrade.com/v1/accounts/xj1Dc18FTqWPqkEEVUr5rw/balance'

In [76]:
resp = session.get(account_balances_url, params = {"instType":instType,"realNAV": False})

In [77]:
resp.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><BalanceResponse><accountId>833535350</accountId><accountType>MARGIN</accountType><optionLevel>LEVEL_4</optionLevel><accountDescription></accountDescription><Cash><fundsForOpenOrdersCash>0.0</fundsForOpenOrdersCash><moneyMktBalance>0.0</moneyMktBalance></Cash><Computed><cashAvailableForInvestment>0.00</cashAvailableForInvestment><cashAvailableForWithdrawal>0.00</cashAvailableForWithdrawal><netCash>-740.450013</netCash><cashBalance>0.0</cashBalance><settledCashForInvestment>0.0</settledCashForInvestment><unSettledCashForInvestment>0.0</unSettledCashForInvestment><fundsWithheldFromPurchasePower>0.0</fundsWithheldFromPurchasePower><fundsWithheldFromWithdrawal>0.0</fundsWithheldFromWithdrawal><OpenCalls><cashCall>0.0</cashCall></OpenCalls><RealTimeValues><totalAccountValue>0.0</totalAccountValue><netMv>-454.22</netMv><netMvLong>0.0</netMvLong></RealTimeValues></Computed></BalanceResponse>'

In [97]:
view_portfolio_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/portfolio.json"

In [98]:
viewp_resp = session.get(view_portfolio_url)

In [80]:
import json

In [99]:
data = json.loads(viewp_resp.text)

In [102]:
viewp_resp.json() == data

True

In [104]:
'PortfolioResponse' in data

True

In [100]:
data

{'PortfolioResponse': {'AccountPortfolio': [{'accountId': '833597000',
    'Position': [{'positionId': 27005131,
      'symbolDescription': 'BR',
      'dateAcquired': -57600000,
      'pricePaid': 0,
      'commissions': 0,
      'otherFees': 0,
      'quantity': 10,
      'positionIndicator': 'TYPE2',
      'positionType': 'LONG',
      'daysGain': -2.7999,
      'daysGainPct': -1.3346,
      'marketValue': 207,
      'totalCost': 0,
      'totalGain': 207,
      'totalGainPct': 0,
      'pctOfPortfolio': 0.0018,
      'costPerShare': 0,
      'todayCommissions': 0,
      'todayFees': 0,
      'todayPricePaid': 0,
      'todayQuantity': 0,
      'lotsDetails': 'https://apisb.etrade.com/v1/accounts/E5Nd4LJBsEi_UyHm4Vio9g/portfolio/27005131',
      'quoteDetails': 'https://apisb.etrade.com/v1/market/quote/BR',
      'Product': {'symbol': 'BR',
       'securityType': 'EQ',
       'expiryYear': 0,
       'expiryMonth': 0,
       'expiryDay': 0,
       'strikePrice': 0,
       'productId'

In [6]:
a = [0, None]

In [11]:
8//10

0